# Week 11 Live Demo — Differentiation, Integration, and ODEs (Refined v3)
**Design goal:** a few realistic physics mini-stories, each unfolding over multiple cells with very explicit comments.


## A) Differentiation from noisy pendulum angle data

In [ ]:
# We simulate a small-angle damped pendulum: θ(t) = θ0 * exp(-β t) * cos(ω_d t)
# and try to estimate angular velocity ω(t) = θ'(t) and angular acceleration α(t) = θ''(t) to estimate energy or damping.
 
# WHAT WE'LL DO:
#  1) Generate noisy θ(t) samples at a realistic frame rate.
#  2) Differentiate directly (and observe the noise explosion).
#  3) Smooth first, then differentiate (much better), and try with higher-order stencils.
 
import numpy as np, matplotlib.pyplot as plt

# --- parameters for the synthetic pendulum ---
θ0   = np.deg2rad(12.0)  # initial amplitude 12° -> radians
β    = 0.15              # light damping [1/s]
ω_d  = 2.0               # damped angular frequency [rad/s]
T    = 10.0              # record for 10 seconds
dt   = 0.02              # 50 Hz camera
t    = np.arange(0.0, T+dt, dt)

# Ground-truth functions (so we can quantify errors later)
def theta_true(t):     return θ0 * np.exp(-β*t) * np.cos(ω_d*t)
def theta_dot_true(t): return θ0 * np.exp(-β*t) * (-β*np.cos(ω_d*t) - ω_d*np.sin(ω_d*t))
def theta_ddot_true(t): 
    return θ0 * np.exp(-β*t) * ((β**2 - ω_d**2)*np.cos(ω_d*t) + 2*β*ω_d*np.sin(ω_d*t))

# Simulated camera noise (≈0.3° RMS)
rng = np.random.default_rng(2)
noise_level = np.deg2rad(0.3)
theta_meas = theta_true(t) + noise_level * rng.normal(size=t.size) # Gaussian noise

# Visualize clean vs measured
plt.figure(figsize=(7.2,3.6))
plt.plot(t, theta_true(t), 'k', lw=2, label='θ true')
plt.plot(t, theta_meas, '.', ms=3, alpha=0.6, label='θ measured (noisy)')
plt.xlabel("time t [s]"); plt.ylabel("angle θ [rad]")
plt.title("Damped pendulum: angle measurements")
plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# NUMERICAL DIFFERENTIATION (NAIVE): central differences directly on the noisy data.
# WHY CENTRAL: it uses points on both sides and has O(dt^2) truncation error (better than forward/backward O(dt)).
# DOWNSIDE: subtracting nearly equal noisy numbers amplifies the noise → very jagged derivative.
 
import numpy as np, matplotlib.pyplot as plt

def central_diff_first(y, dt):
    """Return first derivative using centered stencil; NaN at endpoints where neighbors are missing."""
    dy = np.empty_like(y); dy[:] = np.nan
    dy[1:-1] = (y[2:] - y[:-2]) / (2*dt)
    return dy

omega_est_raw = central_diff_first(theta_meas, dt)   # ω_est from *noisy* θ
omega_true    = theta_dot_true(t)

plt.figure(figsize=(7.2,3.6))
plt.plot(t, omega_true, 'k', lw=2, label='ω true')
plt.plot(t, omega_est_raw, '.', ms=3, alpha=0.6, label='ω estimate (raw θ)')
plt.xlabel("time t [s]"); plt.ylabel("angular velocity ω [rad/s]")
plt.title("Differentiation amplifies noise - naive")
plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# FIX THE NOISE FIRST: smooth θ(t), then differentiate.
# We use a simple *moving average* (box) filter; in a lab you might prefer a low-pass filter.
 
import numpy as np, matplotlib.pyplot as plt

def moving_average(y, window):
    w = int(window)
    if w % 2 == 0: w += 1                 # enforce odd window (symmetric)
    kernel = np.ones(w)/w                 # box filter
    y_pad = np.pad(y, (w//2, w//2), mode='edge')  # simple edge handling
    return np.convolve(y_pad, kernel, mode='valid')

theta_smooth = moving_average(theta_meas, window=9)    # ~0.18 s window at 50 Hz
omega_est_sm = central_diff_first(theta_smooth, dt)    # derivative after smoothing

plt.figure(figsize=(7.2,3.6))
plt.plot(t, omega_true, 'k', lw=2, label='ω true')
plt.plot(t, omega_est_sm, '-', lw=1.3, label='ω (central diff on smoothed θ)')
plt.xlabel("time t [s]"); plt.ylabel("ω [rad/s]")
plt.title("Smoothing first yields a much cleaner derivative")
plt.legend(); plt.tight_layout(); plt.show()

# Quantify improvement with RMS error (ignore NaNs at endpoints)
mask = ~np.isnan(omega_est_raw)
rms_raw = np.sqrt(np.mean((omega_est_raw[mask] - omega_true[mask])**2))
rms_sm  = np.sqrt(np.mean((omega_est_sm[mask]  - omega_true[mask])**2))
print(f"RMS error (raw θ → ω):        {rms_raw}")
print(f"RMS error (smoothed θ → ω):  {rms_sm}")


In [ ]:
# GO ONE STEP FURTHER: use a higher-order stencil on the smoothed data.
# 5-point centered first derivative (O(dt^4)) uses two neighbors on each side.
# Also estimate the second derivative α(t)=θ''(t) with a 3-point stencil.
 
import numpy as np, matplotlib.pyplot as plt

def first_derivative_5pt(y, dt):
    dy = np.empty_like(y); dy[:] = np.nan                 # NaN where stencil doesn't fit
    dy[2:-2] = (-y[4:] + 8*y[3:-1] - 8*y[1:-3] + y[0:-4]) / (12*dt)
    return dy

def second_derivative_3pt(y, dt):
    d2y = np.empty_like(y); d2y[:] = np.nan
    d2y[1:-1] = (y[2:] - 2*y[1:-1] + y[:-2]) / (dt*dt)
    return d2y

omega_est_5pt = first_derivative_5pt(theta_smooth, dt)
alpha_est     = second_derivative_3pt(theta_smooth, dt)
alpha_true    = theta_ddot_true(t)

plt.figure(figsize=(7.2,3.6))
plt.plot(t, omega_true, 'k', lw=2, label='ω true')
plt.plot(t, omega_est_5pt, '-', lw=1.3, label='ω (5-pt on smoothed θ)')
plt.xlabel("time t [s]"); plt.ylabel("ω [rad/s]"); plt.title("Higher-order stencil → more accurate ω")
plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(7.2,3.6))
plt.plot(t, alpha_true, 'k', lw=2, label='α true')
plt.plot(t, alpha_est, '-', lw=1.3, label='α (3-pt on smoothed θ)')
plt.xlabel("time t [s]"); plt.ylabel("α [rad/s²]"); plt.title("Second derivative from data (after smoothing)")
plt.legend(); plt.tight_layout(); plt.show()


## B) Integration as work from nonlinear spring force–displacement data

In [ ]:
# PHYSICS SETUP: a stiffening spring with force F(x) = k x + α x^3.
# The work to compress from x=0 to X is  W = ∫_0^X F(x) dx.
 
import numpy as np, matplotlib.pyplot as plt

k, α = 10.0, 5.0            # toy constants
X    = 1.2                  # final compression [m]

def F(x): return k*x + α*x**3

# Analytic integral for ground truth: 0.5 k X^2 + 0.25 α X^4
W_exact = 0.5*k*X**2 + 0.25*α*X**4
print(f"Exact work (truth) =  {W_exact}")

# Visualize F(x)
xx = np.linspace(0, X, 400)
plt.figure(figsize=(6.6,3.6))
plt.plot(xx, F(xx))
plt.xlabel("compression x [m]"); plt.ylabel("force F [N]")
plt.title("Nonlinear spring force curve")
plt.tight_layout(); plt.show()


In [ ]:
# NUMERICAL integration on [0,X]
# We'll implement Riemann Midpoint, Trapezoid, and Simpson rules and compare them.
 
import numpy as np

def riemann_mid(f, a, b, n):
    x = np.linspace(a, b, n+1); h=(b-a)/n
    xm = 0.5*(x[:-1]+x[1:])           # midpoints of each panel
    return np.sum(f(xm))*h

def trap(f, a, b, n):
    x = np.linspace(a, b, n+1); h=(b-a)/n
    return (h/2)*(f(x[0]) + 2*np.sum(f(x[1:-1])) + f(x[-1]))

def simpson(f, a, b, n):
    if n % 2 == 1: n += 1              # Simpson needs even number of panels
    x = np.linspace(a, b, n+1); h=(b-a)/n
    S = f(x[0]) + f(x[-1]) + 4*np.sum(f(x[1:-1:2])) + 2*np.sum(f(x[2:-2:2]))
    return (h/3)*S

ns = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024] 
print(" n    midpoint         trapezoid        Simpson")
for n in ns:
    mid = riemann_mid(F, 0.0, X, n)
    trp = trap(F, 0.0, X, n)
    smp = simpson(F, 0.0, X, n)
    print(f"{n:3d}  {mid:14.6f}  {trp:14.6f}  {smp:14.6f}")


In [ ]:
# CONVERGENCE STUDY: absolute error vs the number of panels n (log–log plot).
 
import numpy as np, matplotlib.pyplot as plt
ns = np.array([6, 12, 24, 48, 96, 192, 384, 768])
err_mid, err_trp, err_smp = [], [], []
for n in ns:
    err_mid.append(abs(riemann_mid(F,0,X,n) - W_exact))
    err_trp.append(abs(trap(F,0,X,n)        - W_exact))
    err_smp.append(abs(simpson(F,0,X,n)     - W_exact))

plt.figure(figsize=(6.6,3.8))
plt.loglog(ns, err_mid, 'o-', label='midpoint')
plt.loglog(ns, err_trp, 'o-', label='trapezoid')
plt.loglog(ns, err_smp, 'o-', label='Simpson')
plt.xlabel("number of panels n"); plt.ylabel("absolute error |W_n - W_exact|")
plt.title("Work integral: accuracy vs refinement")
plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# ADD SENSOR NOISE: 2% multiplicative noise on F(x) measurements.
 
# Observe how noise reduces the benefit of very high-order rules.
 
import numpy as np, matplotlib.pyplot as plt
rng = np.random.default_rng(3)

def noisy_work(n, noise_frac=0.02):
    # sample at n+1 evenly spaced points
    x = np.linspace(0.0, X, n+1)
    F_meas = F(x) * (1.0 + noise_frac*rng.normal(size=x.size))  # emulate sensor noise
    W_trap = np.trapz(F_meas, x)                                # trapezoid on samples
    # Simpson needs even panels (odd samples). If not, resample by adding one point.
    if n % 2 == 1:
        x = np.linspace(0.0, X, n+2)
        F_meas = F(x) * (1.0 + noise_frac*rng.normal(size=x.size))
    h = (x[-1]-x[0])/(len(x)-1)
    S = F_meas[0] + F_meas[-1] + 4*np.sum(F_meas[1:-1:2]) + 2*np.sum(F_meas[2:-2:2])
    W_simp = (h/3)*S
    return W_trap, W_simp

ns = [12, 24, 48, 96, 192, 384]
err_trap, err_simp = [], []
for n in ns:
    Wt, Ws = noisy_work(n)
    err_trap.append(abs(Wt - W_exact))
    err_simp.append(abs(Ws - W_exact))

plt.figure(figsize=(6.8,3.6))
plt.plot(ns, err_trap, 'o-', label='trapz (noisy) error')
plt.plot(ns, err_simp, 'o-', label='Simpson (noisy) error')
plt.xlabel("panels n"); plt.ylabel("absolute error vs exact")
plt.title("With noisy data, Simpson’s advantage shrinks")
plt.legend(); plt.tight_layout(); plt.show()


## C) ODEs — Cyclotron motion (charged particle in uniform B): compare Euler, Midpoint (RK2), Implicit Euler, Trapezoidal (Heun)

In [ ]:
# PHYSICS MODEL (2D): a particle with charge q and mass m moves in a uniform magnetic field Bz (out of page).
# Lorentz force with E=0 gives  m dv/dt = q v × B,  and x' = v.
# In 2D with B=(0,0,Bz):  v×B = (v_y Bz, -v_x Bz, 0).
# So the ODE system is:
 
#   x' = v_x
#   y' = v_y
#   v_x' =  ω_c v_y         (ω_c = q Bz / m)
#   v_y' = -ω_c v_x
 
# Exact solution is uniform circular motion with radius R = |v0|/ω_c and constant speed.
 
import numpy as np, matplotlib.pyplot as plt

q, m, Bz = 1.0, 1.0, 1.0
ωc = q*Bz/m

# Pack state as Y = [x, y, vx, vy]. Define f(t,Y) = A Y with a constant matrix A (linear system).
A = np.array([[0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 1.0],
              [0.0, 0.0, 0.0,  ωc],
              [0.0, 0.0, -ωc, 0.0]])

def f_lin(t, Y):  # right-hand side f = A Y
    return A @ Y

# Time grid for one cyclotron period
T = 2*np.pi/ωc
h = 0.2                             # step size (we'll vary it later)
t = np.arange(0.0, T+h, h)

# Initial condition: at origin, velocity along +x (speed=1)
Y0 = np.array([0.0, 0.0, 1.0, 0.0])

# --- Stepper methods matching the slides ---
def euler_explicit_step(f, t, Y, h):
    # y_{n+1} = y_n + h f(t_n, y_n)     (simple but can be unstable/inaccurate)
    return Y + h * f(t, Y)

def midpoint_rk2_step(f, t, Y, h):
    # k1 = f(t_n, y_n);  y_{n+1} = y_n + h f(t_n + h/2, y_n + h/2 k1)
    k1 = f(t, Y)
    k2 = f(t + 0.5*h, Y + 0.5*h*k1)
    return Y + h*k2

def heun_trapezoidal_step(f, t, Y, h):
    # Predictor: y* = y_n + h f(t_n, y_n);  Corrector: average slopes
    y_pred = Y + h * f(t, Y)
    return Y + 0.5*h*(f(t, Y) + f(t + h, y_pred))

def euler_implicit_step_linear(A, Y, h):
    # For linear f(Y)=A Y:  (I - h A) y_{n+1} = y_n  -> solve once per step
    I = np.eye(A.shape[0])
    return np.linalg.solve(I - h*A, Y)

# (Optional) Implicit trapezoidal (Crank–Nicolson) for linear systems: (I - h/2 A) y_{n+1} = (I + h/2 A) y_n
def crank_nicolson_step_linear(A, Y, h):
    I = np.eye(A.shape[0])
    return np.linalg.solve(I - 0.5*h*A, (I + 0.5*h*A) @ Y)

def simulate(stepper, Y0, h):
    Y = np.zeros((len(t), 4)); Y[0] = Y0
    for n in range(len(t)-1):
        # Use linear f for Euler/Midpoint/Heun; for implicit variants we call specialized steppers
        if stepper in (euler_implicit_step_linear, crank_nicolson_step_linear):
            Y[n+1] = stepper(A, Y[n], h)
        else:
            Y[n+1] = stepper(f_lin, t[n], Y[n], h)
    return Y

# Run the four methods
Y_eu   = simulate(euler_explicit_step,   Y0, h)
Y_rk2  = simulate(midpoint_rk2_step,     Y0, h)
Y_heun = simulate(heun_trapezoidal_step, Y0, h)
Y_imp  = simulate(euler_implicit_step_linear, Y0, h)            # implicit Euler (linear solve)
Y_cn   = simulate(crank_nicolson_step_linear, Y0, h)             # CN included for reference

# Exact circle for our IC: x(t) = sin(ω t),  y(t) = cos(ω t) - 1  (center at (0,-1), radius 1)
θ = np.linspace(0, 2*np.pi, 600)
x_exact = np.sin(θ)
y_exact = np.cos(θ) - 1.0

# Trajectories
plt.figure(figsize=(6.0,5.6))
plt.plot(Y_eu[:,0],   Y_eu[:,1],   'r.-', label='Explicit Euler')
plt.plot(Y_rk2[:,0],  Y_rk2[:,1],  'g.-', label='Midpoint (RK2)')
plt.plot(Y_heun[:,0], Y_heun[:,1], 'm.-', label='Trapezoidal (Heun)')
plt.plot(Y_imp[:,0],  Y_imp[:,1],  'c.-', label='Implicit Euler')
plt.plot(Y_cn[:,0],   Y_cn[:,1],   'b.-', label='Implicit Trapezoidal (ref)')
plt.plot(x_exact, y_exact, 'k', lw=2, label='exact circle')
plt.axis('equal'); plt.xlabel("x"); plt.ylabel("y")
plt.title("Cyclotron trajectories over one period (fixed h=0.1)")
plt.legend(loc='upper right'); plt.tight_layout(); plt.show()


In [ ]:
# SPEED VS TIME: the exact solution keeps |v| constant = 1.
# Diagnostic: explicit Euler tends to spiral out (energy gain), implicit Euler spirals in (energy loss),
# while trapezoidal/Crank–Nicolson conserve energy much better for this linear system.
 
import numpy as np, matplotlib.pyplot as plt

def speed(Y): return np.sqrt(Y[:,2]**2 + Y[:,3]**2)

plt.figure(figsize=(6.6,3.6))
plt.plot(t, speed(Y_eu),   'r-', label='Explicit Euler |v|')
plt.plot(t, speed(Y_rk2),  'g-', label='Midpoint (RK2) |v|')
plt.plot(t, speed(Y_heun), 'm-', label='Trapezoidal (Heun) |v|')
plt.plot(t, speed(Y_imp),  'c-', label='Implicit Euler |v|')
plt.plot(t, speed(Y_cn),   'b-', label='Implicit Trapezoidal |v|')
plt.axhline(1.0, color='k', lw=1.5, label='true |v| = 1')
plt.xlabel("t"); plt.ylabel("|v|"); plt.title("Speed conservation reveals method/stability behavior")
plt.legend(ncol=2); plt.tight_layout(); plt.show()


In [ ]:
# STEP-SIZE STUDY: final-position error after one period vs h (log–log).
# Smaller h should reduce error; higher-quality methods drop faster.
 
import numpy as np, matplotlib.pyplot as plt

def final_position_error(stepper, h):
    # Integrate one period with step size h and return |(x_T, y_T) - (0,0)|
    ωc = 1.0
    T = 2*np.pi/ωc
    t = np.arange(0.0, T+h, h)
    A = np.array([[0.0, 0.0, 1.0, 0.0],
                  [0.0, 0.0, 0.0, 1.0],
                  [0.0, 0.0, 0.0,  ωc],
                  [0.0, 0.0, -ωc, 0.0]])
    Y = np.zeros((len(t), 4)); Y[0] = np.array([0.0, 0.0, 1.0, 0.0])
    for n in range(len(t)-1):
        if stepper in ('imp', 'cn'):
            if stepper == 'imp':
                Y[n+1] = np.linalg.solve(np.eye(4) - h*A, Y[n])             # implicit Euler
            else:
                Y[n+1] = np.linalg.solve(np.eye(4) - 0.5*h*A, (np.eye(4) + 0.5*h*A) @ Y[n])  # CN
        elif stepper == 'euler':
            Y[n+1] = Y[n] + h*(A @ Y[n])
        elif stepper == 'rk2':
            k1 = A @ Y[n]; k2 = A @ (Y[n] + 0.5*h*k1); Y[n+1] = Y[n] + h*k2
        elif stepper == 'heun':
            y_pred = Y[n] + h*(A @ Y[n]); Y[n+1] = Y[n] + 0.5*h*((A @ Y[n]) + (A @ y_pred))
    return np.hypot(Y[-1,0], Y[-1,1])

hs = np.array([0.4, 0.2, 0.1, 0.05, 0.025])
err_eu  = np.array([final_position_error('euler', h) for h in hs])
err_rk2 = np.array([final_position_error('rk2',   h) for h in hs])
err_heu = np.array([final_position_error('heun',  h) for h in hs])
err_imp = np.array([final_position_error('imp',   h) for h in hs])
err_cn  = np.array([final_position_error('cn',    h) for h in hs])

plt.figure(figsize=(6.8,3.9))
plt.loglog(hs, err_eu,  'o-', label='Explicit Euler')
plt.loglog(hs, err_rk2, 'o-', label='Midpoint (RK2)')
plt.loglog(hs, err_heu, 'o-', label='Trapezoidal (Heun)')
plt.loglog(hs, err_imp, 'o-', label='Implicit Euler')
plt.loglog(hs, err_cn,  'o-', label='Implicit Trapezoidal (ref)')
plt.gca().invert_xaxis()
plt.xlabel("step size h"); plt.ylabel("final position error after 1 period")
plt.title("Convergence with decreasing h (lower is better)")
plt.legend(ncol=2); plt.tight_layout(); plt.show()
